In [2]:
import torchvision
from torchvision.models import vgg19
from torchvision.transforms import transforms
from tqdm import tqdm
import torch
import ImageData
import numpy as np
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
import time

In [3]:
model_name = "VGG"
model_image_size = 224
# vit = models.vit_l_16(models.ViT_L_16_Weights.IMAGENET1K_V1)

In [5]:
class VGG(torch.nn.Module):
    def __init__(self, numClasses):
        super(VGG, self).__init__()
        vgg = vgg19(weights = "DEFAULT")
        self.featureExtractor = vgg.features
        self.avgpool = torch.nn.AdaptiveAvgPool2d((7, 7))
        self.class1, _, _, self.class2, _, _, _  = list(vgg.classifier.children())
        self.class3 = torch.nn.Linear(in_features = 4096, out_features = numClasses)
        self.classifier = torch.nn.Sequential(
            self.class1,
            torch.nn.ReLU(inplace=True),
            self.class2,
            torch.nn.ReLU(inplace=True),
            self.class3
        )

    def forward(self, x):
        #do something 
        x = self.featureExtractor(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)

        return x

In [6]:
model = VGG(4).to(device)
# print(*list(model.children())[:-1])

In [7]:
# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
batch_size = 16
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.RandomResizedCrop(size=(model_image_size, model_image_size), antialias=True), 
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainDataLoader, validDataLoader, testDataLoader = ImageData.getImagesDataloaders("../ArtiFact/", transforms = transform, batchSize=batch_size)

In [11]:
dataset = trainDataLoader.dataset

In [39]:
from torch.utils.data import Subset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split

In [40]:
trainIdxs, _ = train_test_split(range(len(dataset.imagePaths)), train_size=110)

In [46]:
ValIdxs = trainIdxs[100:]
trainIdxs = trainIdxs[:100]

In [48]:
len(trainIdxs)

100

In [49]:
trainSub = Subset(dataset, trainIdxs)
valSub = Subset(dataset, ValIdxs)

In [50]:
trainSubDataloader = DataLoader(trainSub, batch_size=32, shuffle=True)
valSubDataloader = DataLoader(valSub, batch_size=32, shuffle=True)

In [9]:
def evaluate_on_data(model, dataloader, dirty: bool = False):
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        total_loss = 0
        
        num_correct = 0.0
        num_samples = 0.0
        for data in tqdm(dataloader, desc="Eval: "):
            image, label = data
            label = label.to(device)
            image = image.to(device)
            outputs = model(image)
            if dirty:
                label = torch.where(label > 1, torch.tensor(1, dtype = torch.int32).to(device), label)


            loss = criterion(outputs, label)
            total_loss += loss.item()
            argMax = torch.argmax(outputs, 1)
            for i in range(len(label)):
                if label[i] == argMax[i]:
                    num_correct += 1

                num_samples += 1
                    
                
                
    return total_loss / len(dataloader), num_correct / num_samples

In [10]:
num_epochs = 10
count = 0
valid_loss_array = np.zeros(num_epochs)
valid_acc_array = np.zeros(num_epochs)
valid_acc_dirty_array = np.zeros(num_epochs)

train_loss_array = np.zeros(num_epochs)
for epoch in range(num_epochs):
    batch_count = 0
    for data in tqdm(trainDataLoader, desc="Training: "):
        
        image, label = data
        
        label = label.to(device)
        image = image.to(device)

        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        
        count += 1
        # print(loss)
            
        
    valid_loss, valid_acc = evaluate_on_data(model, validDataLoader)
    _, valid_acc_dirty = evaluate_on_data(model, validDataLoader, dirty=True)

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}, Valid Loss: {valid_loss}, Valid ACC: {valid_acc}, Dirty Valid ACC: {valid_acc_dirty}')
    valid_loss_array[epoch] = valid_loss
    train_loss_array[epoch] = loss.item()
    valid_acc_array[epoch] = valid_acc
    valid_acc_dirty_array[epoch] = valid_acc_dirty

#9:53:40
#9:55:28


Training:   0%|          | 0/117213 [00:00<?, ?it/s]

Eval: 100%|██████████| 6170/6170 [13:48<00:00,  7.45it/s]


Epoch [1/10], Training Loss: 0.8625, Valid Loss: 1.0549165871869803, Valid ACC: 0.4855681069854617, Dirty Valid ACC: 0.6085811255762119


Eval: 100%|██████████| 6170/6170 [14:00<00:00,  7.34it/s]


Epoch [2/10], Training Loss: 0.9617, Valid Loss: 1.0550024736739054, Valid ACC: 0.4855681069854617, Dirty Valid ACC: 0.6085811255762119


...

Eval: 100%|██████████| 6170/6170 [08:14<00:00, 12.49it/s]


Epoch [8/10], Training Loss: 0.9442, Valid Loss: 1.054804602849812, Valid ACC: 0.4855681069854617, Dirty Valid ACC: 0.6085811255762119


Eval: 100%|██████████| 6170/6170 [08:10<00:00, 12.58it/s]


Epoch [9/10], Training Loss: 1.0932, Valid Loss: 1.0549815429957219, Valid ACC: 0.4855681069854617, Dirty Valid ACC: 0.6085811255762119


Training:  73%|███████▎  | 85358/117213 [3:56:44<1:30:56,  5.84it/s]

In [20]:
with open(f"{model_name}_valid_loss.npy", 'wb') as f:
    np.save(f, valid_loss_array)
    
with open(f"{model_name}_valid_acc.npy", 'wb') as f:
    np.save(f, valid_acc_array)

with open(f"{model_name}_valid_dirty_acc.npy", 'wb') as f:
    np.save(f, valid_acc_dirty_array)
    
with open(f"{model_name}_train.npy", 'wb') as f:
    np.save(f, train_loss_array)

torch.save(model.state_dict(), f"./results/{model_name}Params.pth")